### Welcome to the prisoner's dilemma game! 

This software is meant for use in experiments, for example in Social Psychology, at desktop machines. Please note that it is not possible to use it as an online version. 
Below you can find code for the single-trial prisoner's dilemma game as well as options for customisation. Opponents are programmed to use one of 5 strategies in each trial (consisting of 7 rounds as default): 

1. Opponent always cooperates
2. Opponent always defects (betrays the participant)
3. Opponent cooperates or defects on a random basis
4. Nice tit for tat: opponent cooperates in the first round, then copies the participant's moves
5. Suspicious tit for tat: opponent defects in the first round, then copies the participant's moves

This notebook will guide you through the code, its objects, functions and specifications and will also inform how to change default values to use it as you need.

The set up of the game is as follows:

> Imagine you are one of two people being questioned about the same crime. They are each talking to the interrogator separately. The interrogator gives each person the same deal: 
they can choose to vouch for the other person’s innocence (COOPERATING) or rat them out (DEFECTING). 
And of course, there’s a twist. If both people cooperate with each other, they’ll each get 3 months off their sentence, but if the first person vouches for the second person, and the second person rats them out, the first person will get no time off their sentence and the second person will get 5 months off their time. Lastly, if they both rat each other out, they each get 1 month off their time.

Here are a few screenshots to see how the game looks like:



![](https://raw.githubusercontent.com/Programming-The-Next-Step/prisonersdilemma/2ee3fdd4045ab1e407f2be0c620d7793bd65817c/screenshot1.png)

![](https://raw.githubusercontent.com/Programming-The-Next-Step/prisonersdilemma/2ee3fdd4045ab1e407f2be0c620d7793bd65817c/screenshot2.png)

![](https://raw.githubusercontent.com/Programming-The-Next-Step/prisonersdilemma/2ee3fdd4045ab1e407f2be0c620d7793bd65817c/screenshot3.png)

These are the modules you will need:

In [ ]:
from psychopy.visual import Window, TextStim, ImageStim
from psychopy.core import wait
from psychopy.event import getKeys, waitKeys
from numpy.random import shuffle
import numpy as np


Next, we want to create the window in which the experiment takes place and open up a log file.

In [ ]:
DISPSIZE = (1000,700)
BGC = 'white'
win = Window(size=DISPSIZE, units='pix', color=BGC, fullscr=False)
backgroundimage = ImageStim(win, 'prisonwall.png', size=(1000, 1000))
#log file
log = open ('logfileprisonersdilemma.txt', 'w')
log.write(' .... ') #insert headers of what should be logged


Now we create the objects we will use. The code here is in the same order as it will be used once we start the game.
We'll start with the objects for introduction and instruction.

In [ ]:
#introduction text
subject_name = 'Kathi'

introstr = f'''
Welcome to the prisoner's dilemma game, {subject_name}!
This game will take about 10 minutes. 
Please wait for further instructions after you've finished.
Press 'space' to continue. 
'''

introtxt = TextStim(win, text=introstr, font='Arial', height=30, color='white', wrapWidth=400)

#instruction text
inststr = '''
Imagine you are one of two people being questioned about the same crime. They are each talking to the interrogator separately. The interrogator gives each person the same deal: 
they can choose to vouch for the other person’s innocence (COOPERATING) or rat them out (DEFECTING). 
And of course, there’s a twist. If both people cooperate with each other, they’ll each get 3 months off their sentence, 
but if the first person vouches for the second person, and the second person rats them out, 
the first person will get no time off their sentence and the second person will get 5 months off their time. 
Lastly, if they both rat each other out, they each get 1 month off their time.

Press 'space' to start the game.
'''

insttxt = TextStim(win, text=inststr, font='Arial', height=25, color='white', wrapWidth=900, pos=(000,-70))


Participants as well as their opponent are represented in the game by avatars. These objects are used for explaining this and defining the image objects. 
These are avatars you can use created with www.hexatar.com, however you are free to use your own. 

These are the avatars for participants:
![participants avatars](https://raw.githubusercontent.com/Programming-The-Next-Step/prisonersdilemma/spitzer/partchoice.png)

These are the avatars representing opponents:
![participants opponents](https://raw.githubusercontent.com/Programming-The-Next-Step/prisonersdilemma/spitzer/opponent1.png)
![participants opponents](https://raw.githubusercontent.com/Programming-The-Next-Step/prisonersdilemma/spitzer/opponent2.png)
![participants opponents](https://raw.githubusercontent.com/Programming-The-Next-Step/prisonersdilemma/spitzer/anonymous.jpg)

Here we define objects during the actual game: instructions about controls, a connecting screen, text to be displayed after every round and a goodbye screen.

In [ ]:
#objects for during the game, in the loop
sentcountp = 35
sentcounto = 35

#info about controls/gameplay, should be displayed on the bottom left during the game
controlinfo = TextStim(win, text='''Press the left arrow to defect and the right arrow to cooperate''', font='Arial', height=20, color='white', wrapWidth=900, 
                       pos=(000, -300))

# txt within game
connectstr = f'Your prison sentence is {sentcountp} months.\n' 
'Your partner`s sentence is {sentcounto} months.\n'
'The authorities are still not sure what to do with the two of you. Therefore, you and your partner\n'
'will be interrogated again by a different police officer. Again you have the choice to cooperate or defect.'

connecttxt = TextStim (win, text=connectstr, font='Arial', height=20, color='black', 
                       wrapWidth=400)             
cooptxt = TextStim (win, text='Your opponent chose to cooperate.', font='Arial', height=20, color='black', 
                    wrapWidth=400)
deftxt = TextStim (win, text='Your opponent chose to defect.', font='Arial', height=20, color='black', 
                   wrapWidth=400)


#goodbye screen
goodbyestr = ('The game is now over.\n' +
'Thank you for playing!\n')
goodbye = TextStim (win, text=goodbyestr, font='Arial', height=40, color='black', 
                    wrapWidth=500)

#lists for during the game
strategy = [1,2,3,4,5]  #random assignment of opponents strategy
shuffle(strategy)
randostrategy = [1,2] #for strategy 3
condlist = [1,2,3] #1 is in-group, 2 is outgroup, 3 is anon
ntrials =  7
oppstrategy = '...' #for updating the sentence counter


Next we define a function for tracking participant's sentence:
During the experiment, we need to update how many months the participant has
to spend in prison now, depending on the choice they made and the choice their
opponent made. For doing this during the trial loop, we write the function
sentencetracker, that checks the participants choice (either cooperating or
defecting) as well as the opponents strategy and updates the sentence based on this and according to the game set up.

In [ ]:
def sentencetracker(sentcounto, sentcountp, response, oppstrategy):
    if response == 'right' and oppstrategy == 'cooperation': #both cooperate
        sentcounto -= 3
        sentcountp -= 3
    elif response == 'right' and oppstrategy == 'defect': #participant wants to cooperate,  but opponent rats them out
        sentcountp -= 5
    elif response == 'left' and oppstrategy == 'cooperation': #participant rats opponent out, but opponent cooperates
       sentcounto -= 5
    else: #response == 'left' & oppstrategy == 'defect': #both defect
        sentcountp -= 1
        sentcounto -= 1
    return sentcounto, sentcountp

Time to start the experiment!
Let's start with the introduction. After that, we explain how it works. We also define the image stimuli for displaying the avatars.

In [ ]:
backgroundimage.draw() #background prisonwall
introtxt.draw()
win.flip()
waitKeys(keyList = ['space']) #wait until participant pressed space to continue
clearEvents()

opponent = ImageStim(win, image="opponent1.png", pos=(300,000))
participant = ImageStim(win, image='partA.png', pos=(-300, 000))

#explaining the game
backgroundimage.draw()
insttxt.draw()
win.flip()
waitKeys(keyList = ['space']) #wait until participant read instructions and pressed space to start the trial loop
strategy = np.random.randint(1,5)
'''Sentence count for the participant and the opponent. You can change this according to your needs, 
however please note that it's 35 because we defined 7 trial rounds and the maximum the 
sentence can change during each round is 5. 7 x 5 is 35, so this is set to prevent the sentence count
from going below 0.
'''
sentcounto = 35
sentcountp = 35

This is the trial loop, it's now set to 7 rounds (see object ntrials = 7 above to change it).
Opponents stick to the same strategy through all 7 rounds. Sentence count as well as strategies are printed in the console as well as logged in the log file.

In [ ]:
i = 0
for i in range(ntrials):
	backgroundimage.draw()
	i += 1
	log.write(str(i))
	log.write('\t')

	#opponent always cooperates
	if strategy == 1:
		opponent.draw()
		participant.draw()
		controlinfo.draw()
		win.flip() 
		log.write('strategy cooperation\t')
		response = waitKeys(keyList=['left', 'right'])
		if response == 'left':
			log.write('defect\n')
		elif response == 'right':
			log.write('cooperate\n')
		cooptxt.draw()
		win.flip()
		wait(4) #seconds
		oppstrategy = 'cooperation'
		print('strategy cooperation')
		sentcounto, sentcountp = sentencetracker(sentcounto, sentcountp, response, oppstrategy) 
		print(sentcounto, sentcountp)
		connectstr = f'''Your prison sentence is {sentcountp} months.
Your partner`s sentence is {sentcounto} months.
The authorities are still not sure what to do with the two of you. 
Therefore, you and your partner will be interrogated again by a different police officer. 
Again you have the choice to cooperate or defect.'''
		connecttxt = TextStim (win, text=connectstr, font='Arial', height=20, color='black', 
					   wrapWidth=400)			               
		connecttxt.draw()
		win.flip()
		wait(7)
		
	
	
	#opponent always defects
	elif strategy == 2:
		opponent.draw()
		participant.draw()
		controlinfo.draw()
		win.flip() 
		log.write('strategy defect\t')
		response = waitKeys(keyList=['left', 'right'])
		if response == 'left':
			log.write('defect\n')
		elif response == 'right':
			log.write('cooperate\n')
		deftxt.draw()
		win.flip()
		wait(4) #seconds
		oppstrategy = 'defect'
		print('strategy defect')
		sentcounto, sentcountp = sentencetracker(sentcounto, sentcountp, response, oppstrategy) 
		print(sentcounto, sentcountp)
		connectstr = f'''Your prison sentence is {sentcountp} months.
Your partner`s sentence is {sentcounto} months.
The authorities are still not sure what to do with the two of you. 
Therefore, you and your partner will be interrogated again by a different police officer. 
Again you have the choice to cooperate or defect.'''
		connecttxt = TextStim (win, text=connectstr, font='Arial', height=20, color='black', 
					   wrapWidth=400)	
		connecttxt.draw()
		win.flip()
		wait(7)
		

	#opponent cooperates or defects on a random basis
	elif strategy == 3:
		rando = np.random.randint(1,3)
		print('rando')
		if rando == 1:
			opponent.draw()
			participant.draw()
			controlinfo.draw()
			win.flip() 
			log.write('strategy random cooperation\t')
			response = waitKeys(keyList=['left', 'right'])
			if response == 'left':
				log.write('defect\n')
			elif response == 'right':
				log.write('cooperate\n')
			cooptxt.draw()
			win.flip()
			wait(4)
			oppstrategy = 'cooperation'
			print('rando coop')
			sentcounto, sentcountp = sentencetracker(sentcounto, sentcountp, response, oppstrategy) 
			print(sentcounto, sentcountp)
			connectstr = f'''Your prison sentence is {sentcountp} months.
Your partner`s sentence is {sentcounto} months.
The authorities are still not sure what to do with the two of you. 
Therefore, you and your partner will be interrogated again by a different police officer. 
Again you have the choice to cooperate or defect.'''
			connecttxt = TextStim (win, text=connectstr, font='Arial', height=20, color='black', 
						   wrapWidth=400)	
			connecttxt.draw()
			win.flip()
			wait(7)
		else: 
			opponent.draw()
			participant.draw()
			controlinfo.draw()
			win.flip()
			log.write('strategy random defect\t')
			wait(4) 
			response = waitKeys(keyList=['left', 'right'])
			if response == 'left':
				log.write('defect\n')
			elif response == 'right':
				log.write('cooperate\n')
			deftxt.draw()
			win.flip()
			wait(4) #seconds
			oppstrategy = 'defect'
			print('rando defect')
			sentcounto, sentcountp = sentencetracker(sentcounto, sentcountp, response, oppstrategy)
			print(sentcounto, sentcountp)
			connectstr = f'''Your prison sentence is {sentcountp} months.
Your partner`s sentence is {sentcounto} months.
The authorities are still not sure what to do with the two of you. 
Therefore, you and your partner will be interrogated again by a different police officer. 
Again you have the choice to cooperate or defect.'''
			connecttxt = TextStim (win, text=connectstr, font='Arial', height=20, color='black', 
						   wrapWidth=400)	
			connecttxt.draw()
			win.flip()
			wait(7)

	elif strategy == 4:
		print('nice tit for tat')
		log.write('strategy nice tit for tat\t')
		if i == 1:
			opponent.draw()
			participant.draw()
			controlinfo.draw()
			win.flip() 
			response = waitKeys(keyList=['left', 'right'])
			if response == 'left':
				log.write('defect\n')
			elif response == 'right':
				log.write('cooperate\n')
			cooptxt.draw()
			win.flip()
			wait(4)
			oppstrategy = 'cooperation'
			print('cooperation')
			sentcounto, sentcountp = sentencetracker(sentcounto, sentcountp, response, oppstrategy) 
			print(sentcounto, sentcountp)
			connectstr = f'''Your prison sentence is {sentcountp} months.
Your partner`s sentence is {sentcounto} months.
The authorities are still not sure what to do with the two of you. 
Therefore, you and your partner will be interrogated again by a different police officer. 
Again you have the choice to cooperate or defect.'''
			connecttxt = TextStim (win, text=connectstr, font='Arial', height=20, color='black', 
						   wrapWidth=400)	
			connecttxt.draw()
			win.flip() 
			wait(7)#seconds
		
		elif i > 1:
			if response == 'right':
				opponent.draw()
				participant.draw()
				controlinfo.draw()
				win.flip() 
				response = waitKeys(keyList=['left', 'right'])
				if response == 'left':
					log.write('defect\n')
				elif response == 'right':
					log.write('cooperate\n')
				cooptxt.draw()
				win.flip()
				wait(4)
				oppstrategy = 'cooperation'
				print('cooperation')
				sentcounto, sentcountp = sentencetracker(sentcounto, sentcountp, response, oppstrategy) 
				print(sentcounto, sentcountp)
				connectstr = f'''Your prison sentence is {sentcountp} months.
Your partner`s sentence is {sentcounto} months.
The authorities are still not sure what to do with the two of you. 
Therefore, you and your partner will be interrogated again by a different police officer. 
Again you have the choice to cooperate or defect.'''
				connecttxt = TextStim (win, text=connectstr, font='Arial', height=20, color='black', 
							   wrapWidth=400)			   
				connecttxt.draw()
				win.flip() 
				wait(7)#seconds
				
			elif response == 'left':
				opponent.draw()
				participant.draw()
				controlinfo.draw()
				win.flip() 
				response = waitKeys(keyList=['left', 'right'])
				if response == 'left':
					log.write('defect\n')
				elif response == 'right':
					log.write('cooperate\n')
				deftxt.draw()
				win.flip()
				wait(4)
				oppstrategy = 'defect'
				print('defect')
				sentcounto, sentcountp = sentencetracker(sentcounto, sentcountp, response, oppstrategy) 
				print(sentcounto, sentcountp)
				connectstr = f'''Your prison sentence is {sentcountp} months.
Your partner`s sentence is {sentcounto} months.
The authorities are still not sure what to do with the two of you. 
Therefore, you and your partner will be interrogated again by a different police officer. 
Again you have the choice to cooperate or defect.'''
				connecttxt = TextStim (win, text=connectstr, font='Arial', height=20, color='black', 
							   wrapWidth=400)	
				connecttxt.draw()
				win.flip() 
				wait(7)#seconds
				
		
	elif strategy == 5:
		print('suspicious tit for tat')
		log.write('strategy suspicious tit for tat\t')
		if i == 1:
			opponent.draw()
			participant.draw()
			controlinfo.draw()
			win.flip() 
			response = waitKeys(keyList=['left', 'right'])
			if response == 'left':
				log.write('defect\n')
			elif response == 'right':
				log.write('cooperate\n')
			deftxt.draw()
			win.flip()
			wait(4)
			oppstrategy = 'defect'
			print('defect')
			sentcounto, sentcountp = sentencetracker(sentcounto, sentcountp, response, oppstrategy) 
			print(sentcounto, sentcountp)
			connectstr = f'''Your prison sentence is {sentcountp} months.
	Your partner`s sentence is {sentcounto} months.
	The authorities are still not sure what to do with the two of you. 
	Therefore, you and your partner will be interrogated again by a different police officer. 
	Again you have the choice to cooperate or defect.'''
			connecttxt = TextStim (win, text=connectstr, font='Arial', height=20, color='black', 
						   wrapWidth=400)	
			connecttxt.draw()
			win.flip() 
			wait(7)#seconds
		
		elif i > 1:
			if response == 'right':
				opponent.draw()
				participant.draw()
				controlinfo.draw()
				win.flip() 
				response = waitKeys(keyList=['left', 'right'])
				if response == 'left':
					log.write('defect\n')
				elif response == 'right':
					log.write('cooperate\n')
				cooptxt.draw()
				win.flip()
				wait(4)
				oppstrategy = 'cooperation'
				print('cooperation')
				sentcounto, sentcountp = sentencetracker(sentcounto, sentcountp, response, oppstrategy) 
				print(sentcounto, sentcountp)
				connectstr = f'''Your prison sentence is {sentcountp} months.
	Your partner`s sentence is {sentcounto} months.
	The authorities are still not sure what to do with the two of you. 
	Therefore, you and your partner will be interrogated again by a different police officer. 
	Again you have the choice to cooperate or defect.'''
				connecttxt = TextStim (win, text=connectstr, font='Arial', height=20, color='black', 
							   wrapWidth=400)			   
				connecttxt.draw()
				win.flip() 
				wait(7)#seconds
				
			elif response == 'left':
				opponent.draw()
				participant.draw()
				controlinfo.draw()
				win.flip() 
				response = waitKeys(keyList=['left', 'right'])
				if response == 'left':
					log.write('defect\n')
				elif response == 'right':
					log.write('cooperate\n')
				deftxt.draw()
				win.flip()
				wait(4)
				oppstrategy = 'defect'
				print('defect')
				sentcounto, sentcountp = sentencetracker(sentcounto, sentcountp, response, oppstrategy) 
				print(sentcounto, sentcountp)
				connectstr = f'''Your prison sentence is {sentcountp} months.
	Your partner`s sentence is {sentcounto} months.
	The authorities are still not sure what to do with the two of you. 
	Therefore, you and your partner will be interrogated again by a different police officer. 
	Again you have the choice to cooperate or defect.'''
				connecttxt = TextStim (win, text=connectstr, font='Arial', height=20, color='black', 
							   wrapWidth=400)	
				connecttxt.draw()
				win.flip() 
				wait(7)#seconds
		
	
		
		
	
	
	#exit early: 
	quitt = getKeys(keyList='q')
	if len(quitt) > 0:
		break

goodbye.draw()
win.flip()
wait(3)
	

win.close()
log.close()

